## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
import os
current_directory = os.getcwd()
print(current_directory) 

C:\Users\clava\World_Weather_Analysis\Vacation_Search


In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\clava\World_Weather_Analysis\Weather_Database\WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Birjand,IR,32.8663,59.2211,49.93,40,0,2.64,clear sky
1,1,Port Alfred,ZA,-33.5906,26.8910,66.94,96,100,6.22,light rain
2,2,Vangaindrano,MG,-23.3500,47.6000,63.30,71,17,11.01,few clouds
3,3,Hilo,US,19.7297,-155.0900,82.13,71,1,11.50,clear sky
4,4,Pimentel,PE,-6.8367,-79.9342,66.49,72,0,20.71,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hilo,US,19.7297,-155.0900,82.13,71,1,11.50,clear sky
5,5,Hithadhoo,MV,-0.6000,73.0833,82.89,75,100,12.95,overcast clouds
6,6,Basco,PH,20.4487,121.9702,79.09,70,100,20.13,overcast clouds
11,11,Alyangula,AU,-13.8483,136.4192,82.27,78,70,3.44,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,75.42,81,86,18.61,light rain
15,15,Bandarbeyla,SO,9.4942,50.8122,76.69,71,3,5.44,clear sky
20,20,Atuona,PF,-9.8000,-139.0333,78.66,67,1,16.53,clear sky
31,31,Kinshasa,CD,-4.3246,15.3215,82.72,78,40,5.75,scattered clouds
43,43,Dakar,SN,14.6937,-17.4441,81.48,94,100,4.61,overcast clouds
45,45,Nileshwar,IN,12.2500,75.1000,77.34,85,57,3.67,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,82.13,clear sky,19.7297,-155.0900,
5,Hithadhoo,MV,82.89,overcast clouds,-0.6000,73.0833,
6,Basco,PH,79.09,overcast clouds,20.4487,121.9702,
11,Alyangula,AU,82.27,broken clouds,-13.8483,136.4192,
12,Rikitea,PF,75.42,light rain,-23.1203,-134.9692,
15,Bandarbeyla,SO,76.69,clear sky,9.4942,50.8122,
20,Atuona,PF,78.66,clear sky,-9.8000,-139.0333,
31,Kinshasa,CD,82.72,scattered clouds,-4.3246,15.3215,
43,Dakar,SN,81.48,overcast clouds,14.6937,-17.4441,
45,Nileshwar,IN,77.34,broken clouds,12.2500,75.1000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
                
        print("Hotel not found... skipping.")
        

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,82.13,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Hithadhoo,MV,82.89,overcast clouds,-0.6000,73.0833,Scoop Guest House
6,Basco,PH,79.09,overcast clouds,20.4487,121.9702,Nathaniel's Lodge
11,Alyangula,AU,82.27,broken clouds,-13.8483,136.4192,Groote Eylandt Lodge
12,Rikitea,PF,75.42,light rain,-23.1203,-134.9692,People ThankYou


In [12]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
3,19.7297,-155.0900
5,-0.6000,73.0833
6,20.4487,121.9702
11,-13.8483,136.4192
12,-23.1203,-134.9692
...,...,...
668,-4.7561,11.8578
675,21.3469,-158.0858
684,-3.6667,152.4333
687,31.3167,-113.5333


In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))